# Data Analysis-- Supervised Learning
* Created on Mon Sep. 29 2021 by Shangying Wang
* Last Modified: April 6, 2023
* this code is used for prediction of the phenotypes from the combinatory motifs
* This code uses the convolutional neural network and LSTM.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import csv
import os
import matplotlib.pyplot as plt
from platform import python_version
print(python_version())

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam
from keras import layers,Sequential
from keras.layers import Lambda
from keras.layers import Input, Dense, Dropout, Flatten, Average, BatchNormalization, LSTM, TimeDistributed
from tensorflow.keras.layers import Conv1D,MaxPool1D, concatenate
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.initializers import RandomNormal,HeNormal,GlorotNormal,HeUniform,LecunNormal,LecunUniform,Orthogonal
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from itertools import product
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
print(tf.__version__)

# For arrayed data analysis

## Load data from csv file
- CSV files available at https://www.science.org/doi/suppl/10.1126/science.abq0225/suppl_file/science.abq0225_data_s1_to_s3.zip
- Change schema to match version used in original commits
- Change motif values to match those used in original commits (i.e., change 17 to 14)

In [ ]:
tf.keras.utils.set_random_seed(1)
train_data=pd.read_csv('science.abq0225_data_s2.csv',encoding= 'unicode_escape',sep=',')
test_data=pd.read_csv('science.abq0225_data_s3.csv',encoding= 'unicode_escape',sep=',')
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)
rename_dict = {'Initial CAR T Cell Number': 'Cell Number',
               'motif i': 'motif',
               'motif j': 'motif.1',
               'motif k': 'motif.2',
               'motif l': 'motif.3',
               'motif m': 'motif.4',
               'Cytotoxicity (Nalm 6 Survival)': 'Nalm 6 Cytotoxicity',
               'Stemness (% IL7Ra+ KLRG1-)': 'IL7RaKLRG1 stemness'}
train_data.rename(columns=rename_dict, inplace=True)
test_data.rename(columns=rename_dict, inplace=True)
train_data.drop([train_data.columns[7]], axis=1, inplace=True)
test_data.drop([test_data.columns[7]], axis=1, inplace=True)
train_data.replace(17, 14, inplace=True)
test_data.replace(17, 14, inplace=True)
test_data

In [ ]:
#data statistics
all_data = pd.concat([test_data, train_data])
max_cell=max(all_data['Cell Number'])
all_data['Cell Number']=all_data['Cell Number']/max_cell
train_data['Cell Number']=train_data['Cell Number']/max_cell
test_data['Cell Number']=test_data['Cell Number']/max_cell
stats_df = all_data.describe()
stats_df

In [ ]:
skew_vals1=all_data['Nalm 6 Cytotoxicity'].skew()
skew_vals1

In [ ]:
skew_vals2=all_data['IL7RaKLRG1 stemness'].skew()
skew_vals2

In [ ]:
plt.figure(figsize=[10,20])
pheno=['IL7RaKLRG1 stemness']
new_data = all_data.copy()
pp=0
plt.subplot(3,2,1)
new_data[pheno[pp]].hist(bins=10)
plt.xlabel('value', fontsize=20)
plt.ylabel('frequency', fontsize=20)
plt.title('before np.log1p', fontsize=20)

plt.subplot(3,2,2)
new_data[pheno[pp]]=new_data[pheno[pp]].apply(np.log1p)
new_data[pheno[pp]].hist(bins=10)
plt.xlabel('value', fontsize=20)
#plt.ylabel('frequency', fontsize=20)
plt.title('after np.log1p', fontsize=20)

In [ ]:
skew_vals2=new_data['IL7RaKLRG1 stemness'].skew()
skew_vals2

In [ ]:
train_data['IL7RaKLRG1 stemness'] = train_data['IL7RaKLRG1 stemness'].apply(np.log1p)
test_data['IL7RaKLRG1 stemness'] = test_data['IL7RaKLRG1 stemness'].apply(np.log1p)

In [ ]:
stats_df = train_data.describe()
stats_df

## Deep Neural Network for Nalm 6 Cytotoxicity

In [ ]:
num_motifs=5
num_class=num_class=len(np.unique(new_data.iloc[:,1:(num_motifs+1)]))
np.unique(new_data.iloc[:,1:(num_motifs+1)])

In [ ]:
max_y=np.max(new_data['Nalm 6 Cytotoxicity'])
max_y

In [ ]:
ICN_train, X_train, Y_train = train_data.iloc[:, :1], train_data.iloc[:,1:(num_motifs+1)], train_data['Nalm 6 Cytotoxicity']/max_y
ICN_test, X_test, Y_test = test_data.iloc[:, :1], test_data.iloc[:,1:(num_motifs+1)], test_data['Nalm 6 Cytotoxicity']/max_y

### One-hot encoding for 14 linear motifs: 

In [ ]:
X_train_channel=to_categorical(X_train, num_classes=num_class)
X_test_channel=to_categorical(X_test, num_classes=num_class)
print(np.shape(X_test_channel)) #3D tensor with shape (batch_size, steps, features/channels)

In [ ]:
output_dim = 1
batch_size = 10

In [ ]:
stats_df = Y_train.describe()
stats_df

## GridSearchCV--search for the optimal hyperparameters  

In [ ]:
np.shape(np.reshape(X_train_channel, [np.shape(X_train_channel)[0], -1]))

In [ ]:
#In order to tune the parameters, define deep NN structure as a tuning function
def create_model(filters=32, kernel_size=3, LSTM_units=4, dropout_rate1=0.2, fc_nodes=50):
    #seperate inputs:
    # create model
    input_all=Input(shape=(76,),name='input_all')
    input1=Lambda(lambda x: x[:,:-1])(input_all)
    input_position = tf.keras.layers.Reshape((num_motifs, num_class), input_shape=(75,))(input1)
    input_ICN = Lambda(lambda x: tf.expand_dims(x[:,-1],-1))(input_all) # (None,1)
                           
    x=Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu', input_shape=(num_motifs, num_class))(1.0*input_position)
    x=LSTM(LSTM_units,return_sequences=True, dropout=dropout_rate1)(x)#return_sequences=True,
    x = Flatten()(x)
    model1 = keras.Model(inputs=input_all, outputs=x)


    # combine the output of the two branches
    combined = concatenate([model1.output, input_ICN])
    # apply a FC layer and then a regression prediction on the
    # combined outputs

    z = Dense(fc_nodes, activation='relu')(combined)
    z = Dropout(0.5)(z)
    z = Dense(1, activation='relu')(z)

    # our model will accept the inputs of the two branches and
    # then output a single value
    model = keras.Model(inputs=input_all, outputs=z)

    # Compile model
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# create model
model_LSTM = KerasRegressor(build_fn=create_model)
# define the grid search parameters
# layer1_units = [16, 32, 64, 128]
# layer2_units=[3, 4, 5, 6]
# dropout_rate1=[0.1, 0.2, 0.3, 0.4, 0.5]
# dropout_rate2=[0.1, 0.2, 0.3, 0.4, 0.5]
filters = [10, 20, 50]
kernel_size = [2, 3, 4, 5]
LSTM_units=[2, 4, 8]
dropout_rate1=[0.0, 0.1, 0.2]
fc_nodes=[6, 14, 64]
param_grid = dict(filters=filters, kernel_size=kernel_size, LSTM_units=LSTM_units, dropout_rate1=dropout_rate1, fc_nodes=fc_nodes)
LSTM_search = GridSearchCV(estimator=model_LSTM, param_grid=param_grid, cv=10, verbose=2)


In [ ]:
## merge inputs
combine_input_train = np.concatenate([np.reshape(X_train_channel, [np.shape(X_train_channel)[0], -1]), ICN_train], axis=1)
combine_input_test = np.concatenate([np.reshape(X_test_channel, [np.shape(X_test_channel)[0], -1]), ICN_test], axis=1)

In [ ]:
## grid search to find the best model （This steps takes a long time to run)
tf.keras.utils.set_random_seed(2)
best_model = LSTM_search.fit(combine_input_train, Y_train,  batch_size=batch_size,  epochs=1200, verbose=0)

In [ ]:
print(best_model.best_estimator_.get_params())

In [ ]:
print('Best Hyperparameters: %s' % best_model.best_estimator_.get_params())

In [ ]:
print('Best Score: %s' % best_model.best_score_)

In [ ]:
# Rerun the gbrt with the best combination of hyperparameters
#large max_depth to prevent overfitting
model_LSTM_best = KerasRegressor(build_fn=create_model, filters=64, LSTM_units=3, dropout_rate1=0.2, fc_nodes=50)
model_LSTM_best.fit(combine_input_train, Y_train, batch_size=batch_size,  epochs=1200, verbose=1)
pred_train = model_LSTM_best.predict(combine_input_train)*max_y
pred_test = model_LSTM_best.predict(combine_input_test)*max_y

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test*max_y, pred_test))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test*max_y, pred_test))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test*max_y, pred_test)))

In [ ]:
title_list=['Nalm 6 Cytotoxicity','IL7RaKLRG1 stemness']
def do_plot(ax_row, pred_train, pred_test):
    i=0
    gt=Y_train*max_y
    ax_row[0].scatter(gt,pred_train)
    xmin=min(min(gt),min(pred_train))
    xmax=max(max(gt),max(pred_train))
    xline=np.linspace(xmin,xmax,10)
    ax_row[0].plot(xline,xline,color='red')
    ax_row[0].set_xlabel('Ground Truth (Training)')
    ax_row[0].set_ylabel('Predictions (Training)')
    correlation_matrix = np.corrcoef(gt, pred_train)
    corr = correlation_matrix[0,1]
    r_squared_train = corr**2
    ax_row[0].set_title(title_list[i]+'\n'+'R^2='+str(r_squared_train)[:5])
    ax_row[0].axis('square')

    gt=Y_test*max_y
    ax_row[1].scatter(gt,pred_test)
    xmin=min(min(gt),min(pred_test))
    xmax=max(max(gt),max(pred_test))
    xline=np.linspace(xmin,xmax,10)
    ax_row[1].plot(xline,xline,color='red')
    ax_row[1].set_xlabel('Ground Truth (Test)', fontsize=20)
    ax_row[1].set_ylabel('Predictions (Test)', fontsize=20)
    correlation_matrix = np.corrcoef(gt, pred_test)
    corr = correlation_matrix[0,1]
    r_squared_test = corr**2
    ax_row[1].set_title(title_list[i]+'\n'+'R^2='+str(r_squared_test)[:5])
    plt.axis('square')
    
    return r_squared_train, r_squared_test

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=[15,70])
do_plot(axs, pred_train, pred_test)

## Train models using hyperparameters in supplementary materials
### See Table S1 of https://www.science.org/doi/suppl/10.1126/science.abq0225/suppl_file/science.abq0225_sm.pdf

In [ ]:
hyperparameter_set = [
    [20, 5, 4, 0., 64, ],
    [10, 2, 8, 0., 16, ],
    [20, 4, 8, 0.2, 4, ],
    [10, 2, 4, 0., 64, ],
    [20, 4, 2, 0., 16, ],
    [20, 2, 4, 0., 4, ],
    [10, 5, 4, 0., 16, ],
    [20, 5, 4, 0., 4, ],
    [20, 2, 4, 0., 4, ],
    [50, 5, 4, 0., 4, ],
]
tf.keras.utils.set_random_seed(3)
fig, axs = plt.subplots(len(hyperparameter_set), 2, figsize=[15,70])
for i, (filters, kernel_size, LSTM_units, dropout_rate, fc_nodes) in enumerate(hyperparameter_set):
    # Rerun the gbrt with the best combination of hyperparameters
    #large max_depth to prevent overfitting
    model_LSTM_best = KerasRegressor(build_fn=create_model, filters=filters, LSTM_units=LSTM_units, dropout_rate1=dropout_rate, fc_nodes=fc_nodes)
    model_LSTM_best.fit(combine_input_train, Y_train, batch_size=batch_size,  epochs=1200, verbose=0)
    pred_train = model_LSTM_best.predict(combine_input_train)*max_y
    pred_test = model_LSTM_best.predict(combine_input_test)*max_y
    r_squared_train, r_squared_test = do_plot(axs[i], pred_train, pred_test)
    print(i+1, filters, kernel_size, LSTM_units, dropout_rate, fc_nodes, '{:.3f}'.format(r_squared_train), '{:.3f}'.format(r_squared_test))
    model_LSTM_best.model.save(f'saved_model/position_encoding_arrayed_data_04062023_CNN_LSTM_Cytotoxicity_{i+1}.h5')